## 证照识别V2.1
1. 由于现在的证照的与原来的证照的图片有点不一样, 上次的的营业执照的图片有一个长方形的图片在外面,可以很好的识别出来这个,
2. 现在的营业执照上面也有一个长方形的外框,不过在这个外壳上面有一个国徽.
3. 如果和以前一样识别那种长方形的框,这个框会识别不出来.
4. 觉得采用新的识别方式,先识别出有字的地方,然后在识别出对应的字
5. 参考:https://blog.csdn.net/dcrmg/article/details/52095508
6. 步骤如下:
    1. 先识别二维码的信息
    2. 在识别具体的信息
        1. 通过最大池化，保留图片的最大属性
        1. 原图像大小调整，提高运算效率
        2. 转化为灰度图
        3. 高斯平滑滤波
        4. 求得水平和垂直方向灰度图像的梯度差,使用Sobel算子
        5. 均值滤波，消除高频噪声
        6. 二值化
        7. 闭运算，填充条形码间隙
        8. 腐蚀，去除孤立的点
        9. 膨胀，填充条形码间空隙，根据核的大小，有可能需要2~3次膨胀操作
        10. 通过findContours找到条形码区域的矩形边界


In [207]:
import cv2
import numpy as np
import pyzbar.pyzbar as pz

import torch
from torch import nn
from torchvision import transforms

from PIL import Image, ImageChops
import matplotlib.pyplot as plt 


import pytesseract
pytesseract.pytesseract.tesseract_cmd = "C:/Program Files (x86)/Tesseract-OCR/tesseract.exe"

import imutils
from imutils.perspective import four_point_transform


In [208]:
img_url = r"F:\PYcode\Coding\ic\license\source_img\4.jpg"

In [209]:
# 四边形矫正
def change_size(image):
    # 读取图片
    width1, height1 = image.shape[:2]
    # 转灰色
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # 高斯滤波
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # 自适应二值化方法
    blurred=cv2.adaptiveThreshold(blurred,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,51,2)
    # 找到边框
    edged = cv2.Canny(blurred, 10, 100)
    # 找到矩阵
    cnts = cv2.findContours(edged, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if imutils.is_cv2() else cnts[1]
    docCnt = None
    
    if len(cnts) > 0:
#         将轮廓按大小降序排序
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
#         # 对排序后的轮廓循环处理
        for c in cnts:
#             # 获取近似的轮廓
         

            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            # 如果近似轮廓有四个顶点，那么就认为找到了图片

            if len(approx) == 4:
                docCnt = approx
                break
    newimage = image.copy()
    for i in docCnt:
        #circle函数为在图像上作图，新建了一个图像用来演示四角选取
        cv2.circle(newimage, (i[0][0],i[0][1]), 10, (255, 0, 0), -1)
    # 使用transform 做映射
    paper = four_point_transform(image, docCnt.reshape(4, 2))
    outimg = paper[int(paper.shape[0]*0.02):int(paper.shape[0]*0.98), int(paper.shape[1]*0.02):int(paper.shape[1]*0.98)]
    
    cv2.imwrite("resoutlicence.jpg", outimg)
    return outimg

In [210]:
def get_contours(img):
    # 高斯模糊一下
    img_g = cv2.GaussianBlur(img,(25,25), 0)   

    # 均值滤波，消除高频噪声 blur

#     #  sobelx = cv2.Sobel(frame,cv2.CV_64F,1,0,ksize=5)
    imageX = cv2.Sobel(img_g, cv2.CV_64F,1,0,ksize=5)
    imageY = cv2.Sobel(img_g, cv2.CV_64F,0,1,ksize=5)
    # convertScaleAbs(imageX16S,imageSobelX,1,0);
    imageSobelX = cv2.convertScaleAbs(imageX, 1.0)
    imageSobelY = cv2.convertScaleAbs(imageY, 1.0)
    imageSobelOut = imageSobelX + imageSobelY
#     cv2.imwrite("bur.jpg", imageSobelOut)
    bur = cv2.blur(imageSobelOut, (25,25))
    bur = cv2.cvtColor(bur, cv2.COLOR_BGR2GRAY)
#     # 二值化 threshold
    retval,threshold = cv2.threshold(bur,125,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
#     # 闭运算，
    kernel = np.ones((5,5),np.uint8)  
    imgc = cv2.morphologyEx(threshold,cv2.MORPH_CLOSE, kernel)
    # 腐蚀，去除孤立的点 erode
    kernel2 = np.ones((15,15),np.uint8) 
    imge = cv2.erode(imgc, kernel2, iterations = 2) 
#     # 膨胀，填充字体之间空隙，根据核的大小，有可能需要2~3次膨胀操作 dilate
    imgd = cv2.dilate(imge, kernel, iterations = 9)
#     cv2.imwrite("imgd.jpg", imgd)

    # 通过findContours找到条形码区域的矩形边界
    img, contours, hierarchy = cv2.findContours(imgd,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) 
    cv2.imwrite("img.jpg", imgd)

    return contours

In [219]:
# 对数据进行筛选
def filter_data(dataes):
    xy_sort_data = []
    for data in dataes:
        rect = cv2.minAreaRect(data)
        box = cv2.boxPoints(rect)
        x, y, w, h = cv2.boundingRect(box)
        if w/h > 0.75:
            xy_sort_data.append([x, y, w, h])
    xy_sort_data.sort(key = lambda x: x[1])
#     xy_sort_data.sort(key = lambda x: x[2])

    return xy_sort_data
list_data = filter_data(contours)

In [220]:
# 读取图片
source_img = cv2.imread(img_url, cv2.IMREAD_UNCHANGED)
# 改变图片的形状
img_changed = change_size(source_img)
# 获取到有四边形的点
contours = get_contours(img_changed)
# 参看有多少个形状
list_data = filter_data(contours)

In [221]:
i =1
for x, y, w, h in list_data:
    cv2.rectangle(img_changed,(x,y),(x+w,y+h),(i*4,i*3,255-i*3),1) 
    cv2.putText(img_changed, str(i), (x,y), font, 0.5, (11,0,255), 2, cv2.LINE_AA)
    i += 1          
cv2.imwrite("img_changed.jpg", img_changed)

True

In [216]:
for contour in contours:
    rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(rect)
    x, y, w, h = cv2.boundingRect(box)
    cv2.rectangle(img_changed,(x,y),(x+w,y+h),(0,0,255),1) 
    
cv2.imwrite("img_changed.jpg", img_changed)
cv2.imshow("sss", img_changed)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [225]:
i =1
xlist = []
for x, y, w, h in list_data:
    xlist.append(y)
    arr_mean = np.mean(xlist)
    arr_var = np.var(xlist)
    arr_std = np.std(xlist,ddof=1)
    print(i)

    print("平均值为：%f" % arr_mean)
    print("方差为：%f" % arr_var)
    print("标准差为:%f" % arr_std)
    print("*"*20)
#     cv2.rectangle(img_changed,(x,y),(x+w,y+h),(i*4,i*3,255-i*3),1) 
#     cv2.putText(img_changed, str(i), (x,y), font, 0.5, (11,0,255), 2, cv2.LINE_AA)
    i += 1

1
平均值为：419.000000
方差为：0.000000
标准差为:nan
********************
2
平均值为：460.500000
方差为：1722.250000
标准差为:58.689863
********************
3
平均值为：567.333333
方差为：23974.888889
标准差为:189.637373
********************
4
平均值为：623.000000
方差为：27277.500000
标准差为:190.709203
********************
5
平均值为：657.600000
方差为：26610.640000
标准差为:182.382291
********************
6
平均值为：681.666667
方差为：25071.555556
标准差为:173.452779
********************
7
平均值为：709.142857
方差为：26019.551020
标准差为:174.230143
********************
8
平均值为：762.000000
方差为：42324.250000
标准差为:219.933107
********************
9
平均值为：814.000000
方差为：59253.555556
标准差为:258.186464
********************
10
平均值为：860.200000
方差为：72538.160000
标准差为:283.897791
********************
11
平均值为：898.090909
方差为：80300.991736
标准差为:297.205469
********************
12
平均值为：930.500000
方差为：85163.083333
标准差为:304.803513
********************
13
平均值为：972.615385
方差为：99896.544379
标准差为:328.969993
********************
14
平均值为：1009.500000
方差为：110447.250000
标准差为:344.881418
*******************

import numpy as np
arr = [1,2,3,4,5,6]
#求均值
arr_mean = np.mean(arr)
#求方差
arr_var = np.var(arr)
#求标准差
arr_std = np.std(arr,ddof=1)
print("平均值为：%f" % arr_mean)
print("方差为：%f" % arr_var)
print("标准差为:%f" % arr_std)

# 123


In [145]:
img_g = cv2.cvtColor(img_changed, cv2.COLOR_BGR2GRAY)
#     retval,threshold = cv2.threshold(img_g,125,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)
threshold = cv2.adaptiveThreshold(img_g, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 111, 1)
img_g = cv2.medianBlur(threshold,ksize =7)
# img_g = cv2.Laplacian(img_g, cv2.CV_64F)
img_g = cv2.medianBlur(img_g,ksize =7)

In [161]:
# cv2.imshow("sss", img_g)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

cv2.imwrite("img_changed.jpg", img_changed)

True

In [79]:
# 使用原始图片的大小做一个判断
area = int(img_changed.shape[0]*img_changed.shape[1] / 10000)
font = cv2.FONT_HERSHEY_SIMPLEX 

print(area)
print("*"*20)

xy_sort_list = []
box_list = []
i = 1
rect = cv2.minAreaRect(contour)
box = cv2.boxPoints(rect)
x, y, w, h = cv2.boundingRect(box)


for contour in contours:
#     rect = cv2.minAreaRect(contour)
#     print(rect)
#     if cv2.contourArea(contour) > area:
        
    x, y, w, h = cv2.boundingRect(contour)
#         if w/h> 0.75:
    xy_sort_list.append([x, y, w, h])
    cv2.rectangle(img_changed,(x,y),(x+w,y+h),(i*4,i*3,255-i*3),1) 

# xy_sort_list.sort(key = lambda x:x[1])
        
# for x, y, w, h in xy_sort_list:
#     cv2.rectangle(img_changed,(x,y),(x+w,y+h),(i*4,i*3,255-i*3),1) 
    cv2.putText(img_changed, str(i), (x,y), font, 0.5, (11,0,255), 2, cv2.LINE_AA)
    i += 1          
cv2.imwrite("img_changed.jpg", img_changed)
# for i in range(len(contours)):
#     print(cv2.contourArea(contours[i]))
#     x, y, w, h = cv2.boundingRect(contours[i])
#     print(w*h)
#     print("*"*20)

# i = 1

    
# for i in range(len(xy_sort_list)-1):
#     x, y, w, h = xy_sort_list[i]
#     x_1, y_1, w_1, h_1 = xy_sort_list[i+1]
#     if w/h > 0.75:
#         print(((y_1 - y)**2 +(x_1 -x)**2)**0.5)
    

1269
********************


True

In [50]:
cv2.imshow("sss", img_changed)
cv2.waitKey(0)
cv2.destroyAllWindows()

# cv2.imwrite("finish.jpg", img1)

In [53]:
# 识别时候用的图片
img2 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
_, img3 = cv2.threshold(img2,125,255, cv2.THRESH_BINARY)
# img3 = cv2.morphologyEx(threshold,cv2.MORPH_CLOSE, (3,3))
# img3 = cv2.erode(img3, (5,5), iterations = 2) 
# img3 = cv2.dilate(imge, (3,3), iterations = 5)

In [54]:
if pz.decode(img1):    
    for i in pz.decode(img1):
        print(i.data.decode('utf-8'))
        print(i.rect)
        print(i.polygon)

统一社会信用代码：91510100MA63DEFJ00
企业注册号：510109001599590
名称：成都西辰易督科技有限公司
登记机关：成都市高新工商行政管理局
登记日期：2019-01-23
企业信用信息公示系统网址：http://www.cdgs.gov.cn  http://gsxt.cdcredit.gov.cn
Rect(left=928, top=3282, width=372, height=374)
[Point(x=928, y=3649), Point(x=1295, y=3656), Point(x=1300, y=3287), Point(x=935, y=3282)]


# 结束

In [73]:
# 图片转灰色
img1= cv2.imread(source_img)
# img = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)

In [75]:
img1 = change_size(img1)

In [76]:
# 高斯平滑滤波
img = cv2.GaussianBlur(img1,(25,25), 0)
# bur = cv2.blur(img, (7,7))

In [77]:
# 求得水平和垂直方向灰度图像的梯度差,使用Sobel算子
#  sobelx = cv2.Sobel(frame,cv2.CV_64F,1,0,ksize=5)
imageX = cv2.Sobel(img, cv2.CV_64F,1,0,ksize=5)
imageY = cv2.Sobel(img, cv2.CV_64F,0,1,ksize=5)
# convertScaleAbs(imageX16S,imageSobelX,1,0);
imageSobelX = cv2.convertScaleAbs(imageX, 1.0)
imageSobelY = cv2.convertScaleAbs(imageY, 1.0)
imageSobelOut = imageSobelX + imageSobelY
# 相减不成功，相加成功

In [78]:
# 均值滤波，消除高频噪声 blur
bur = cv2.blur(imageSobelOut, (25,25))
bur = cv2.cvtColor(bur, cv2.COLOR_BGR2GRAY)

In [79]:
# 二值化 threshold
retval,threshold = cv2.threshold(bur,125,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)

In [80]:
# 闭运算，
kernel1 = np.ones((5,5),np.uint8)  
imgc = cv2.morphologyEx(threshold,cv2.MORPH_CLOSE, kernel1)

In [81]:
# 腐蚀，去除孤立的点 erode
kernel2 = np.ones((15,15),np.uint8) 
imge = cv2.erode(imgc, kernel2, iterations = 2) 

In [82]:
# 膨胀，填充条形码间空隙，根据核的大小，有可能需要2~3次膨胀操作 dilate
imgd = cv2.dilate(imge, kernel1, iterations = 9)

In [83]:
# 通过findContours找到条形码区域的矩形边界
# contours, hierarchy = cv2.findContours(imgd, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
_, contours, hierarchy = cv2.findContours(imgd,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) 

In [102]:
box_list = []
xy_list = []
area = int(img1.shape[0]*img1.shape[1]/900)
i = 1
for contour in contours:
    if cv2.contourArea(contour) > area:
        x, y, w, h = cv2.boundingRect(box)
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
#         cv2.drawContours(img1, [box], 0, (0, 0, 255), 1)
        box_list.append(box)
        x, y, w, h = cv2.boundingRect(box)
        xy_list.append([x, y, w, h])
        resout = pytesseract.image_to_string(img1[ y:(y+h), x:(x+w)], lang='chi_sim')
#         if resout:
        cv2.imwrite(r"img_re\aa%s.jpg"%(i), img1[ y:(y+h), x:(x+w)])
#         print("img%s"%(i), resout)
        cv2.imwrite(r"img_re\change.jpg", img1)
        
    else:
        continue
    i += 1

In [105]:
box_list = []
xy_list = []
area = int(img1.shape[0]*img1.shape[1]/900)
i = 1
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    xy_list.append([x, y, w, h])
    xy_list.sort(key = lambda x:x[1])
    
for i in range(len(xy_list)-1):
    x, y, w, h = xy_list[i]
    x_1, y_1, w_1, h_1 = xy_list[i+1]
    if w/h > 0.75:
        print(((y_1 - y)**2 +(x_1 -x)**2)**0.5 )
    
    
#     if cv2.contourArea(contour) > area:
#         print(cv2.contourArea(contour))
#         rect = cv2.minAreaRect(contour)
#         box = cv2.boxPoints(rect)
#         x, y, w, h = cv2.boundingRect(box)
#         xy_list.append([x, y, w, h])
#         xy_list.sort(key = lambda x:x[1])
        
#         if w/h >= 0.75:
#             print(w)
        
# #             rect = cv2.minAreaRect(contour)
# #             box = cv2.boxPoints(rect)
# #             box = np.int0(box)
# #     #         cv2.drawContours(img1, [box], 0, (0, 0, 255), 1)
# #             box_list.append(box)
#             x, y, w, h = cv2.boundingRect(box)
#             xy_list.append([x, y, w, h])
#             resout = pytesseract.image_to_string(img1[ y:(y+h), x:(x+w)], lang='chi_sim')
#     #         if resout:
#             cv2.imwrite(r"img_re\aa%s.jpg"%(i), img1[ y:(y+h), x:(x+w)])
#             print("img%s"%(i), resout)
#             cv2.imwrite(r"img_re\change.jpg", img1)
        
#     else:
#         continue
#     i += 1

208.25465180878913
103.1746092796091
882.0459171721163
434.07372645669307
884.0361983538909
771.0045395456501
269.6405014088203
97.082439194738
1495.707524885798
158.00316452527144
541.1561696959576
1421.48689758295
431.14034837857616
176.91240770505613
1592.9601376054582
434.1658669218482
169.85287751463028
616.4357225210103
433.2597373400856
176.13914953808538
618.4698860898565
595.4838368923207
613.7165469498113
140.00357138301865
149.00335566691106
145.34441853748632
184.53184007102948
643.4757182675971
439.11388044560834
283.0017667789373
159.31415505221122
286.29530209208815
613.4924612413749
288.0433995077825
156.1569723067145
295.28630174798155
445.0280890011326
1591.4072388926727
2118.2570665525936
143.0034964607509
289.0069203323685
2263.0019885099528
1666.000300120021
154.02921800749363
1802.7792432796646
1514.6233855318621
573.4003836761883
135.1332675546625
267.00187265260894
138.01449199268893
1473.3397435757986
1573.513584307425


In [93]:
xy_list.sort(key = lambda x:x[1])
font = cv2.FONT_HERSHEY_SIMPLEX 
for i in range(len(xy_list)):
    x,y,w,h = xy_list[i]
#     x_1,y_1,w_1,h_1 = xy_list[i+1]
    if w/h >= 0.75:
        

#     cv2.rectangle(img1,(xy_list[i][0], xy_list[i][1]),(xy_list[i][0]+xy_list[i][2],xy_list[i][1]+xy_list[i][3]),(i*4,i*3,255-i*3),1)
    
        cv2.rectangle(img1,(x,y),(x+w,y+h),(i*4,i*3,255-i*3),1) 

        cv2.putText(img1, str(i), (x,y), font, 0.5, (11,0,255), 2, cv2.LINE_AA)
#     print(i*4,i*3,255-i*2)
    


In [94]:
cv2.imshow("sss", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [95]:
cv2.imwrite("finish.jpg", img1)

True

[[1955, 3716, 171, 81],
 [383, 3647, 1100, 155],
 [1852, 3534, 107, 103],
 [1990, 3532, 102, 104],
 [1723, 3531, 101, 101],
 [1588, 3525, 103, 102],
 [2146, 3393, 478, 447],
 [666, 3071, 406, 397],
 [2767, 2737, 115, 158],
 [2826, 2689, 59, 92],
 [1024, 2636, 100, 109],
 [1178, 2633, 1562, 263],
 [2844, 2632, 43, 46],
 [581, 2629, 124, 112],
 [870, 2627, 119, 123],
 [727, 2626, 123, 120],
 [2845, 2593, 37, 37],
 [2767, 2525, 46, 191],
 [1176, 2489, 700, 95],
 [731, 2484, 112, 92],
 [1026, 2471, 105, 117],
 [870, 2464, 114, 124],
 [582, 2459, 121, 127],
 [1180, 2322, 477, 85],
 [894, 2309, 82, 102],
 [735, 2299, 107, 106],
 [1018, 2298, 112, 121],
 [579, 2288, 128, 124],
 [1209, 2157, 574, 90],
 [1025, 2143, 111, 103],
 [880, 2133, 114, 118],
 [731, 2132, 123, 117],
 [591, 2131, 116, 114],
 [1189, 1993, 148, 87],
 [594, 1969, 544, 122],
 [1198, 1836, 1143, 98],
 [1023, 1816, 117, 115],
 [590, 1801, 119, 124],
 [1193, 1673, 1484, 98],
 [1024, 1656, 114, 108],
 [590, 1644, 121, 122],
 [27

In [45]:
box_list[2]

array([[1989, 3640],
       [1989, 3532],
       [2091, 3532],
       [2091, 3640]], dtype=int64)

In [46]:
img2gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

imageVar = cv2.Laplacian(img2gray, cv2.CV_64F).var()
cv2.imshow('imageVar',imageVar)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [47]:
x, y, w, h = cv2.boundingRect(box_list[1])
x, y, w, h

(382, 3642, 1102, 165)

In [48]:
resout = pytesseract.image_to_string(img1[y:(y+h),x:(x+w)])

In [49]:
imgxx = img1[860:1250, 900:2700]

In [50]:
# equ = cv2.equalizeHist(imgg) # 只能传入灰度图
# res = np.hstack((imgg,equ))  # 图像列拼接（用于显示）
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 结束
------
***


In [ ]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cl1 = clahe.apply(imgg)

cv2.imshow('img',imgg)
cv2.imshow('cl1',cl1)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import numpy as np
import cv2
 
def ComputeHist(img):
    h,w = img.shape
    hist, bin_edge = np.histogram(img.reshape(1,w*h), bins=list(range(257)))
    return hist
    
def ComputeMinLevel(hist, rate, pnum):
    sum = 0
    for i in range(256):
        sum += hist[i]
        if (sum >= (pnum * rate * 0.01)):
            return i
            
def ComputeMaxLevel(hist, rate, pnum):
    sum = 0
    for i in range(256):
        sum += hist[255-i]
        if (sum >= (pnum * rate * 0.01)):
            return 255-i
            
def LinearMap(minlevel, maxlevel):
    if (minlevel >= maxlevel):
        return []
    else:
        newmap = np.zeros(256)
        for i in range(256):    #获取阈值外的像素值 i< minlevel，i> maxlevel
            if (i < minlevel):
                newmap[i] = 0
            elif (i > maxlevel):
                newmap[i] = 255
            else:
                newmap[i] = (i-minlevel)/(maxlevel-minlevel) * 255
        return newmap
        
def CreateNewImg(img):
    h,w,d = img.shape
    newimg = np.zeros([h,w,d])
    for i in range(d):
        imgmin = np.min(img[:,:,i])
        imgmax = np.max(img[:,:,i])
        imghist = ComputeHist(img[:,:,i])
        minlevel = ComputeMinLevel(imghist, 8.3, h*w)
        maxlevel = ComputeMaxLevel(imghist, 2.2, h*w)
        newmap = LinearMap(minlevel,maxlevel)
        if (newmap.size ==0 ):
            continue
        for j in range(h):
            newimg[j,:,i] = newmap[img[j,:, i]]
    return newimg


if __name__ == '__main__':
    img1 = cv2.imread(r"F:\PYcode\Coding\ic\license\source_img\6.jpg")
    img = cv2.resize(img1,(563,750))    
    newimg = CreateNewImg(img)
    cv2.imshow('original_img', img)
    cv2.imshow('new_img', newimg/255)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [ ]:
imga  = cv2.imread(r'F:\PYcode\faceinfo\tree.jpg')
B,G,R = cv2.split(imga)

In [ ]:
mb = B.mean()
mg = G.mean()
mr = R.mean()
kb = (mb + mg + mr) / (3 * mb)
kg = (mb + mg + mr) / (3 * mg)
kr = (mb + mg + mr) / (3 * mr)
B = B * kb
G = G * kg
R = R * kr

In [ ]:
new_img = cv2.merge([R,G,B])

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread(r"F:\PYcode\Coding\ic\license\source_img\6.jpg",0)
# img = cv2.resize(img,(563,750))

img = cv2.medianBlur(img,5)

ret,th1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,11,2)
th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)

titles = ['Original Image', 'Global Thresholding (v = 127)',
            'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
images = [img, th1, th2, th3]

for i in range(4):
    plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()


In [ ]:
cv2.imwrite("th3.jpg",th3)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 颜色各通道的直方图
def calcAndDrawHist(image, color):  
    hist= cv2.calcHist([image], [0], None, [256], [0.0,255.0])  
    minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(hist)  
    histImg = np.zeros([256,256,3], np.uint8)  
    hpt = int(0.9* 256);  

    for h in range(256):  
        intensity = int(hist[h]*hpt/maxVal)  
        cv2.line(histImg,(h,256), (h,256-intensity), color)  

    return histImg
import numpy as np
b, g, r = cv2.split(imga)  
histImgB = calcAndDrawHist(b, [255, 0, 0])  
histImgG = calcAndDrawHist(g, [0, 255, 0])  
histImgR = calcAndDrawHist(r, [0, 0, 255])  

cv2.imshow("histImgB", histImgB)  
cv2.imshow("histImgG", histImgG)  
cv2.imshow("histImgR", histImgR)  
cv2.imshow("Img", img)  
cv2.waitKey(0)  
cv2.destroyAllWindows() 

In [21]:
import cv2  
  
img = cv2.imread(r'F:\PYcode\faceinfo\4.jpg')  
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)  
ret, binary = cv2.threshold(gray,127,255,cv2.THRESH_BINARY)  
  
_, contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)  
cv2.drawContours(img,contours,-1,(0,0,255),1)  
  
cv2.imshow("img", img)  
cv2.waitKey(0)  
cv2.destroyAllWindows() 

In [25]:

for contour in contours:
    #计算一个简单的边界框
    x,y,w,h = cv2.boundingRect(contour)
    #画出边界框
    cv2.rectangle(img, (x,y), (x+w, y+h),(0,0,255),2)
    #计算包围目标的最小矩形区域
    rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(img, [box], 0, (255, 255, 255), 2)
    #cv2.minEnclosingCircle函数返回一个二元组，第一个元组为圆心坐标，第二个为半径
    (x0,y0), radius = cv2.minEnclosingCircle(contour)
    
    center = (int(x0), int(y0))
    radius = int(radius)
    img = cv2.circle(img, center, radius, (0,255,0))

cv2.drawContours(img, contours, -1, (255,0,0), 1)
cv2.imwrite("findcontour.jpg",img)
cv2.imshow("img", img)  
cv2.waitKey(0)  
cv2.destroyAllWindows() 